## Predicting the stock market

In this project, we'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the **S&P500 Index** from **1950** to **2015**. The dataset is stored in sphist.csv downloaded from kaggle.

The columns of the dataset are:

- **Date** -- The date of the record.
- **Open** -- The opening price of the day (when trading starts).
- **High** -- The highest trade price during the day.
- **Low** -- The lowest trade price during the day.
- **Close** -- The closing price for the day (when trading is finished).
- **Volume** -- The number of shares traded.
- **Adj Close** -- The daily closing price, adjusted retroactively to include any corporate actions.

We'll be using this dataset to develop a predictive model. We'll train the model with data from **1950-2012**, and try to make predictions from **2013-2015**.

In [33]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

#settings
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
#load data
sphist = pd.read_csv("sphist.csv")

sphist.shape
sphist.head()

(16590, 7)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


There are no null values in the data which is good. The **Date** column is in string. Let's convert it into date type so we can perform date operation using it. Once we convert we shall sort the dataframe on the date column in ascending order.

In [26]:
sphist['Date'] = pd.to_datetime(sphist['Date'])

sphist = sphist.sort_values("Date")
sphist = sphist.reset_index(drop=True)

As stock market data is seqential and we cannot train our model to use future data for training past rows. Injecting future knowledge will make our model look good when you're training and testing it, but will make it fail in the real world. 

We shall generate few indicator columns that will capture the average price using historical data. This will make predictions more accurate.

Here are indicators that are interesting to generate for each row:

- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.

In [27]:
#generate indicator columns
days = [5,30,365]
for day in days:
    #generate column name
    col_name = "avg_" + str(day) + "_days"
    sphist[col_name] = 0
    for idx, row in sphist.iterrows():
        if idx >= day:
            #calculate past n days "Close" price average and assign to new column
            avg = sphist['Close'][idx-day:idx].mean()
            sphist.loc[idx, col_name] = avg

Let's view the head of the data to check the newly created columns.

In [28]:
sphist.head(8)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_days,avg_30_days,avg_365_days
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.0,0.0
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.0,0.0
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.0,0.0
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.0,0.0
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.0,0.0
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.0,0.0
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.0,0.0
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.0,0.0


New columns look good. But we need to remove the rows with "0" values as we are computing indicators with historical data. And the rows with value "0" don't capture this due to insufficient number of days to compute the indicator.

So let's drop rows with '0' in column "avg_365_days" which will also remove rows with '0' in other 2 columns.

Note: Checked for any other rows with value '0' ohter than first 365. There are none.

In [29]:
sphist = sphist[sphist['avg_365_days'] != 0]

sphist.shape

(16225, 10)

We are good to build a model using this data now. Let's split the data into train and test sets .

As mentioned earlier we will use data prior to 2013 to make predictions in 2013-15. Let's split the data accordingly. We use Linear Regression to build the model.

Note: We will leave out all of the original columns (Close, High, Low, Open, Volume, Adj Close, Date) when training the model. These all contain knowledge of the future that we don't want to feed the model.

In [32]:
#split data to trian and test sets
train = sphist[sphist['Date'] < datetime(year=2013, month=1, day=1)]
test = sphist[sphist['Date'] >= datetime(year=2013, month=1, day=1)]

#build model
features = ['avg_5_days', 'avg_30_days', 'avg_365_days']
target = 'Close'

ln = LinearRegression()
ln.fit(train[features], train[target])

#predictions
train_pred = ln.predict(train[features])
test_pred = ln.predict(test[features])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Here we shall chose MAE(Mean Absolute Error) as our metric to evaluate the model. As it will show us how "close" we were to the price in intuitive terms. MSE (Mean Squared Error), is an alternative that is more commonly used, but makes it harder to intuitively tell how far off we are from the true price because it squares the error.

In [40]:
print("MAE train:", mean_absolute_error(train[target], train_pred))
print("MAE test:", mean_absolute_error(test[target], test_pred))

#stats of Close column
test['Close'].describe()

MAE train: 4.980534340688775
MAE test: 16.14243964355504


count     739.000000
mean     1874.890338
std       192.389058
min      1457.150024
25%      1697.450012
50%      1920.239990
75%      2052.165039
max      2130.820068
Name: Close, dtype: float64

Our test error in 16.14 which is pretty good. This means our prediction of Close price is off by only 16.14 dollars. With the mean value of 1874 and standard deviation of 192 for the test dataset, 16.14 is a small number.

However there is a scope to improve the model. We can see that the model is over fit. The model is trained well with MAE of 4.98 but when it faultered to generalize with such accuracy.

--------------------------------

### Ideas to improve further
There's a lot of improvement still to be made on the indicator side, we can think of better indicators that we could use for prediction. We can also make significant structural improvements to the algorithm, and pull in data from other sources.

- Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what we'd do if we were trading using the algorithm.

- We can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.

- We can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before, and the amount of Twitter activity around certain stocks.

- We can also make the system real-time by writing an automated script to download the latest data when the market closes, and make predictions for the next day.

- Finally, We can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second by second predictions. This will require obtaining more data, though. We could also make predictions for individual stocks instead of the S&P500.